In [ ]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist_m as mnistm
from data import mnist
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from data.tasks import load_task
from experiments.training import *
from experiments.SL_bound import *
#from experiments.DA_bound import *
from bounds.bounds import *
from util.kl import *
from util.misc import *
from util.batch import *
from results.plotting import *

# Hyper-parameters
task = 4
seed = 69105
n_classifiers = 2
delta=0.05 ## what would this be?   
binary=True
bound='germain'
architecture="lenet"
epsilons=[0.01]#,0.001]0.15,0.1,0.05
#[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
# 0.6-0.9 left on chexpert
alphas=[0.3]#,0.4,0.5,0.6,0.7,0.8,0.9],0.6,0.7,0.8,0.9
sigmas=[[3,2]]#,[3,2],[3,3]

project_folder2 = "/cephyr/users/adambre/Alvis/"
project_folder="/cephyr/NOBACKUP/groups/snic2021-23-538/mnist_transfer/"

# Alvis params

In [ ]:
project = 'SNIC2021-7-83'
username = 'adambre'
job = 'batch_bound_single.sbatch'

In [ ]:
!squeue -u $username

In [ ]:
!cat $job

In [ ]:
print('Iterating over experiments...\n')

fids = []

os.makedirs(project_folder+'logs', exist_ok=True)


np.random.seed(seed)
# for s in range(10):
#     seed+=1
#     print(seed)
for alpha in alphas:

    print("alpha:"+str(alpha))

    for epsilon in epsilons:
        print("  epsilon:"+str(epsilon))
        for sigma in sigmas:    
            print("    sigma:"+str(sigma))
            arg_list = get_job_args(task, bound=bound, alpha=alpha, sigma=sigma,
                                    epsilon=epsilon, binary=binary, n_classifiers=n_classifiers,architecture=architecture)
#             for a in arg_list:#[5:]:
#                 ckpt = os.path.splitext(os.path.basename(a['posterior_path']))[0]
#                 print(ckpt)
            #sys.exit(-1)
            for a in arg_list:#[5:]:   

                ckpt = os.path.splitext(os.path.basename(a['posterior_path']))[0]
                fid = project_folder+'logs/batch_t-%d_r-%d_a-%.4f_s-%d%d_e-%.4f_d-%.4f_b-%d_n-%d_B-%s_c-%s' % \
                    (task, seed, alpha, sigma[0], sigma[1], epsilon, delta, binary, n_classifiers, bound, ckpt)

                sigstr = '"%d.%d"' % (sigma[0], sigma[1])
                exp = 'task=%d,seed=%d,alpha=%.4f,sigma="%s",epsilon=%.4f,delta=%.4f,'% (task, seed, alpha, sigstr, epsilon, delta)\
                        +'binary=%d,nclassifiers=%d,bound=%s,prior=%s,posterior=%s,architecture=%s' % (binary, n_classifiers, bound, a['prior_path'], a['posterior_path'],architecture)

                prior_path = a['prior_path']
                output = !sbatch -o "$fid"-%j.out -e "$fid"-%j.err -A $project --export="$exp" $job

                jobid = int(output[0].split(' ')[-1])
                fid = fid+'-%s' % jobid

                fids.append(fid)

In [ ]:
print(fids)

In [ ]:
!squeue -u $username

"""
to remove all jobs queued

squeue -u adambre | awk '{if ($1!=JOBID) && ($1!=160158) {print $1}}' | tail -n +2 |xargs scancel


"""

In [ ]:
for fid in fids: 
    print('------')
    print(fid)
    !tail -n 10 "$fid".out
    !tail -n 5 "$fid".err
    print(' \n\n')